<h2> Imports & Configuration </h2>

In [1]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
import time

In [2]:
from pyspark.sql.types import IntegerType
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

In [3]:
import cml.data_v1 as cmldata

# Sample in-code customization of spark configurations
#from pyspark import SparkContext
#SparkContext.setSystemProperty('spark.executor.cores', '1')
#SparkContext.setSystemProperty('spark.executor.memory', '2g')

CONNECTION_NAME = "paul-aug26-aw-dl"
conn = cmldata.get_connection(CONNECTION_NAME)
spark = conn.get_spark_session()

sc = spark.sparkContext
sc.setLogLevel("ERROR")

import os
print("https://spark-"+os.environ["CDSW_ENGINE_ID"]+"."+os.environ["CDSW_DOMAIN"])

Setting spark.hadoop.yarn.resourcemanager.principal to pauldefusco


Spark Application Id:spark-eb08f77ec4814e08b05832343025ec36


In [4]:
# spark.conf.set("spark.sql.shuffle.partitions", "3")
spark.conf.set("spark.sql.adaptive.enabled", "false")

# Join Skews

In [5]:
STORAGE = "s3a://paul-aug26-buk-a3c2b50a/data/pdefusco"

#s3a://go01-demo/datalake/pdefusco/transactions/

transactions_file = "/transactions/*"
customer_file = "/pii/piiData.csv"

df_transactions = spark.read.parquet(STORAGE + transactions_file)
df_customers = spark.read.option("header",True).csv(STORAGE + customer_file)

In [6]:
print(df_transactions.count())
print(df_customers.count())

11000000


20000


In [7]:
df_transactions.printSchema()
df_transactions.show(5, False)

root
 |-- credit_card_number: string (nullable = true)
 |-- credit_card_provider: string (nullable = true)
 |-- transaction_type: string (nullable = true)
 |-- event_ts: timestamp (nullable = true)
 |-- longitude: float (nullable = true)
 |-- latitude: float (nullable = true)
 |-- transaction_currency: string (nullable = true)
 |-- transaction_amount: decimal(10,0) (nullable = true)



+------------------+--------------------+----------------+-------------------+---------+--------+--------------------+------------------+
|credit_card_number|credit_card_provider|transaction_type|event_ts           |longitude|latitude|transaction_currency|transaction_amount|
+------------------+--------------------+----------------+-------------------+---------+--------+--------------------+------------------+
|12345678901234    |VISA 16 digit       |purchase        |2023-09-03 00:33:00|-86.0    |40.3963 |CHF                 |20137             |
|12345678901234    |Mastercard          |purchase        |2023-09-24 02:44:00|-83.0    |41.3963 |MEX                 |21871             |
|12345678901234    |VISA 13 digit       |cash_advance    |2023-12-27 10:19:00|-68.0    |48.3963 |USD                 |29623             |
|12345678901234    |JCB 16 digit        |purchase        |2023-01-05 12:49:00|-124.0   |24.3963 |EUR                 |369               |
|12345678901234    |JCB 15 digit  

In [8]:
df_customers.printSchema()
df_customers.show(5, False)

root
 |-- name: string (nullable = true)
 |-- address: string (nullable = true)
 |-- email: string (nullable = true)
 |-- aba_routing: string (nullable = true)
 |-- bank_country: string (nullable = true)
 |-- account_no: string (nullable = true)
 |-- int_account_no: string (nullable = true)
 |-- swift11: string (nullable = true)
 |-- credit_card_number: string (nullable = true)

+-----------+--------------------------+-------------------------+-----------+------------+------------------+----------------------+-----------+------------------+
|name       |address                   |email                    |aba_routing|bank_country|account_no        |int_account_no        |swift11    |credit_card_number|
+-----------+--------------------------+-------------------------+-----------+------------+------------------+----------------------+-----------+------------------+
|null       |9738 Alejandro Glens      |null                     |null       |null        |null              |null         

In [10]:
df_transactions.groupBy("credit_card_number").count().orderBy("credit_card_number").show()

+------------------+--------+
|credit_card_number|   count|
+------------------+--------+
|    12345678901234|10000000|
|        1942892530|       1|
|        2045911182|       1|
|        2045911183|       1|
|        2045911184|       1|
|        2045911185|       1|
|        2045911186|       1|
|        2045911187|       1|
|        2045911188|       1|
|        2045911189|       1|
|        2045911190|       1|
|        2045911191|       1|
|        2045911192|       1|
|        2045911193|       1|
|        2045911194|       1|
|        2045911195|       1|
|        2045911196|       1|
|        2045911197|       1|
|        2045911198|       1|
|        2045911199|       1|
+------------------+--------+
only showing top 20 rows



In [11]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", -1)

In [12]:
df_txn_details = (
    df_transactions.join(
        df_customers,
        on="credit_card_number",
        how="inner"
    )
)

In [13]:
start_time = time.time()
df_txn_details.count()
print(f"time taken: {time.time() - start_time}")

time taken: 38.24294400215149


# Using AQE

In [13]:
spark.conf.set("spark.sql.adaptive.enabled", "true")
spark.conf.set("spark.sql.adaptive.skewedJoin.enabled", "true")

In [15]:
df_txn_details = (
    df_transactions.join(
        df_customers,
        on="credit_card_number",
        how="inner"
    )
)

In [16]:
start_time = time.time()
df_txn_details.count()
print(f"time taken: {time.time() - start_time}")

time taken: 7.173134803771973


# Using Broadcast Joins

In [14]:
# 10MB = 10485760 Bytes
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 31457280)

In [15]:
df_txn_details = (
    df_transactions.join(
        F.broadcast(df_customers),
        on="credit_card_number",
        how="inner"
    )
)

In [16]:
start_time = time.time()
df_txn_details.count()
print(f"time taken: {time.time() - start_time}")

time taken: 23.771111965179443


In [18]:
# spark.stop()